In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import hog
import matplotlib.pyplot as plt
import json

Loads an image from the specified path.

In [ ]:
# Load the image
image_path = "Preprocessed\image_path"
image = cv2.imread(image_path)

In [ ]:
Converts preprocessed image to grayscale (gray) using OpenCV (cv2 library).

In [ ]:
# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

Otsu's thresholding method (cv2.THRESH_OTSU) on the grayscale image (gray) to segment the image into foreground and background regions. The result is stored in segmented.

In [ ]:
# Perform thresholding for segmentation
_, segmented = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

Applies the Canny edge detection algorithm on the segmented image to detect edges. Parameters 100 and 200 represent the low and high thresholds for edge detection.

In [ ]:
# Apply Canny edge detection on the segmented image
edges = cv2.Canny(segmented, 100, 200)

HOG Parameters: Sets up parameters (hog_params) for computing HOG features:

    orientations: Number of orientation bins.
    pixels_per_cell: Size (in pixels) of a cell.
    cells_per_block: Number of cells in each block.
    block_norm: Block normalization method.
    visualize: Whether to return a visualization image (hog_image).
    transform_sqrt: Whether to apply power law compression to normalize gradients.

In [ ]:
# Define parameters for HOG
hog_params = {
    'orientations': 9,
    'pixels_per_cell': (8, 8),
    'cells_per_block': (2, 2),
    'block_norm': 'L2-Hys',
    'visualize': True,
    'transform_sqrt': True
}

HOG Calculation: Computes HOG features (hog_features) from the edges (edges) using the specified parameters.

In [ ]:
# Compute HOG features and visualization
hog_features, hog_image = hog(edges, **hog_params)

Implements the Improved Local Ternary Patterns (ILTP) algorithm:

    *Purpose: Computes a binary code representation of local image patterns based on pixel intensity differences.
    
    *Parameters: Takes an input image (grayscale) and a threshold value to determine pattern coding.
    
    *Returns: Returns the ILTP image (iltp_image) where each pixel represents a computed pattern code.

In [ ]:
# Function to compute Improved Local Ternary Patterns (ILTP)
def iltp(image, threshold=5):
    rows, cols = image.shape
    iltp_image = np.zeros((rows, cols), dtype=np.int32)

    for i in range(1, rows-1):
        for j in range(1, cols-1):
            center = image[i, j]
            code = 0
            for k in range(8):
                row_offset = [-1, -1, 0, 1, 1, 1, 0, -1]
                col_offset = [0, 1, 1, 1, 0, -1, -1, -1]
                neighbor = image[i + row_offset[k], j + col_offset[k]]
                code <<= 3
                if neighbor > center + threshold:
                    code |= 2
                elif neighbor < center - threshold:
                    code |= 0
                else:
                    code |= 1
            iltp_image[i, j] = code
    return iltp_image

In [ ]:
# Compute ILTP features
iltp_image = iltp(edges)

Computes color histograms for each channel (blue, green, red) of the preprocessed image (image) in BGR format:

    *Parameters: bins specifies the number of bins for histogram calculation (default is 256).
    
    *Returns: Returns flattened histograms (hist_b, hist_g, hist_r) for each color channel.
        - hist_b (numpy.ndarray): Blue channel histogram.
        - hist_g (numpy.ndarray): Green channel histogram.
        - hist_r (numpy.ndarray): Red channel histogram.

In [ ]:
# Compute color histograms for each channel
def compute_color_histograms(image, bins=256):
    hist_b = cv2.calcHist([image], [0], None, [bins], [0, 256])
    hist_g = cv2.calcHist([image], [1], None, [bins], [0, 256])
    hist_r = cv2.calcHist([image], [2], None, [bins], [0, 256])
    return hist_b.flatten(), hist_g.flatten(), hist_r.flatten()

hist_b, hist_g, hist_r = compute_color_histograms(image)

Converts multidimensional feature arrays (hog_features, iltp_image, edges, hist_b, hist_g, hist_r) into flattened arrays (*_flat) suitable for storage.

In [ ]:
# Flatten the features for storage
hog_features_flat = hog_features.flatten()
iltp_features_flat = iltp_image.flatten()
edges_flat = edges.flatten()
hist_b_flat = hist_b.flatten()
hist_g_flat = hist_g.flatten()
hist_r_flat = hist_r.flatten()

Creates a dictionary (features) to store flattened features as lists (tolist()) for JSON serialization.

In [ ]:
# Create a dictionary for storing features
features = {
    'hog_features': hog_features_flat.tolist(),
    'iltp_features': iltp_features_flat.tolist(),
    'canny_edges': edges_flat.tolist(),
    'color_histogram_blue': hist_b_flat.tolist(),
    'color_histogram_green': hist_g_flat.tolist(),
    'color_histogram_red': hist_r_flat.tolist()
}

Ensures the output directory (/mnt/data/) exists or creates it if not (os.makedirs(output_dir, exist_ok=True)).

In [ ]:
# Ensure the directory exists
output_dir = '/mnt/data/'
os.makedirs(output_dir, exist_ok=True)

Saves the extracted features (features dictionary) to a JSON file (segmentation_features.json) in the specified directory (output_dir) using Python's json.dump().

In [ ]:
# Save the features to a JSON file
output_file = os.path.join(output_dir, 'segmentation_features.json')
with open(output_file, 'w') as f:
    json.dump(features, f)

print("Features saved successfully.")